#### Data Reading, Cleaning, and Organizing

In [2]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

**Reading the Data**

In [7]:
# reading aisles dataset
aisles = pd.read_csv('data/aisles.csv')

In [8]:
# reading department dataset
department = pd.read_csv('data/departments.csv')

In [9]:
# reaading random 5% of order_prior dataset 
p = 0.05
order_prior = pd.read_csv('data/order_products__prior.csv', header = 0,  skiprows=lambda i: i>0 and random.random() > p)

In [10]:
# reading random 5% of the orders dataset
p = 0.05
orders = pd.read_csv('data/orders.csv', header = 0,  skiprows=lambda i: i>0 and random.random() > p)

In [11]:
# reading in products data
products = pd.read_csv('data/products.csv')

In [12]:
# reading random 5% of the train order dataset
p = 0.05
order_train = pd.read_csv('data/order_products__train.csv', header = 0,  skiprows=lambda i: i>0 and random.random() > p)

In [13]:
aisles.head(2)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses


In [14]:
department.head(2)

,department_id,department
0,1,frozen
1,2,other


In [15]:
order_prior.head(2)

,order_id,product_id,add_to_cart_order,reordered
0,2,9327,3,0
1,3,17704,3,1


In [17]:
orders.head(2)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,473747,1,prior,3,3,12,21.0
1,1416320,16,test,7,0,13,7.0


In [20]:
products.head(2)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13


In [21]:
order_train.head(2)

,order_id,product_id,add_to_cart_order,reordered
0,1,11109,2,1
1,36,34497,6,1


In [16]:
order_prior.shape

(1623277, 4)

In [18]:
orders.shape

(170092, 7)

In [19]:
order_train.shape

(69048, 4)

In [22]:
order_prior.shape

(1623277, 4)

In [23]:
# check the orders dataset for null values
orders.isnull().sum()

order_id                      0
user_id                       0
eval_set                      0
order_number                  0
order_dow                     0
order_hour_of_day             0
days_since_prior_order    10246
dtype: int64

Missing values in the 'days since prior order' column mean that these customers did not make prior purchases.

In [24]:
orders.dropna(axis = 0, inplace = True) # delete rows with missing values

**Split Orders Dataframe into Prior, Train, and Test Dataframes**

In [25]:
orders['eval_set'].value_counts(normalize = True) # breakdown of orders dataframe

prior    0.935970
train    0.040376
test     0.023654
Name: eval_set, dtype: float64

In [26]:
# create filter to separate test orders
orders_test_filtered = orders['eval_set'] == 'test'

In [27]:
# apply filter to orders dataframe 
orders_test = orders[orders_test_filtered]

In [28]:
# save test orders to csv file for modeling 
orders_test.to_csv('data/orders_test.csv')

In [29]:
# create filter to separate prior orders
orders_filtered_prior = orders['eval_set'] == 'prior'

In [30]:
# apply filter to a dataframe
orders_prior_details = orders[orders_filtered_prior]

In [31]:
# create filter to separate train orders
orders_filtered_train = orders['eval_set'] == 'train' 

In [32]:
# apply filter to a dataframe
orders_train_details = orders[orders_filtered_train]

**Merge products and order_prior dataframes**

In [36]:
prod_orders_prior = pd.merge(products, order_prior,  on = 'product_id') # marge products and orders

In [37]:
prod_orders_prior.head(2)

,product_id,product_name,aisle_id,department_id,order_id,add_to_cart_order,reordered
0,1,Chocolate Sandwich Cookies,61,19,25365,9,1
1,1,Chocolate Sandwich Cookies,61,19,33675,10,1


In [38]:
prod_orders_prior.shape # check if all rows remain

(1623277, 7)

In [39]:
len(prod_orders_prior['order_id'].unique())

1176991

**Merge products and order_train dataframes**

In [40]:
prod_orders_train = pd.merge(products, order_train,  on = 'product_id') # merge products and train orders

In [42]:
prod_orders_train.shape

(69048, 7)

**Concat prod_orders_train and prod_order_prior dataframes**

In [43]:
orders_train_prior = pd.concat([prod_orders_train, prod_orders_prior], ignore_index = True, sort = False)

In [44]:
orders_train_prior.shape

(1692325, 7)

**Add quantity column**

In [45]:
# create quantity column reflecting number of products
orders_train_prior.loc[orders_train_prior['product_name'].notna(), 'quantity'] = 1

In [46]:
# lowercase product names
orders_train_prior['product_name'] = orders_train_prior['product_name'].str.lower()

In [47]:
# take out commas in the product name
orders_train_prior['product_name'] = orders_train_prior['product_name'].str.replace(",","")

In [48]:
# save resulting dataframe into a csv file
orders_train_prior.to_csv('data/orders_train_prior.csv', index = False)

**Merge prod_orders_prior, orders_prior_details dataframes**

In [49]:
all_orders_prior = pd.merge(prod_orders_prior, orders_prior_details, on = 'order_id')

In [54]:
# lowercase product names
all_orders_prior['product_name'] = all_orders_prior['product_name'].str.lower()

In [55]:
# take out commas in the product names
all_orders_prior['product_name'] = all_orders_prior['product_name'].str.replace(",","")

In [56]:
# save the dataframe into a csv file
all_orders_prior.to_csv('data/all_orders_prior.csv', index = False)

**Merge prod_orders_train, prod_orders_prior and all orders dataframes**

In [58]:
all_orders_train = pd.merge(prod_orders_train, orders_train_details, on = 'order_id')

In [59]:
# lowercase product names
all_orders_train['product_name'] = all_orders_train['product_name'].str.lower()

In [60]:
# take out commas in the product name column
all_orders_train['product_name'] = all_orders_train['product_name'].str.replace(",","")

In [61]:
# save the dataframe into a csv file
all_orders_train.to_csv('data/all_orders_train.csv', index = False)

**Concat all train and prior orders**

In [62]:
all_orders = pd.concat([all_orders_train, all_orders_prior], ignore_index = True, sort = False)

**Add quantity column to all_orders dataframe**

In [64]:
# create quantity column reflecting number of products
all_orders.loc[all_orders['product_name'].notna(), 'quantity'] = 1

**Merge aisles, department, and all_orders dataframes**

In [65]:
all_orders = pd.merge(all_orders, aisles, on = 'aisle_id')

In [66]:
all_orders = pd.merge(all_orders, department, on = 'department_id')

In [68]:
all_orders['product_name'] = all_orders['product_name'].str.lower() # lowercase product names

In [69]:
all_orders['product_name'] = all_orders['product_name'].str.replace(",","") # take out commas in the product names

In [70]:
# save the dataframe to a csv file
all_orders.to_csv('data/all_orders.csv', index = False) 